In [33]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering

# Quer aprender como eu faço um projeto de Data Science da ideia ao produto final? Acesse CursoDeDataScience.com

https://www.kaggle.com/lukeimurfather/tweets

In [2]:
tweets = pd.read_csv("nCoV_tweets.csv", index_col=0, parse_dates=['dt'])

In [129]:
docs = ["curso de data, dAta science", 
        "tutorial de data analysis", 
        "não sei mais o que escrever analysis"]

In [133]:
bag_of_words_transformer = CountVectorizer(binary=False, analyzer='word', stop_words=['de', 'que'], ngram_range=(1,1), min_df=2)
mx = bag_of_words_transformer.fit_transform(docs).todense()
terms = bag_of_words_transformer.get_feature_names()
pd.DataFrame(mx, columns=terms, index=docs)


#lowercase
#analyzer='char'
#strip_accents='unicode'
#binary=True

#stop_words=['de']
#ngram_range=(1,1)
#min_df

#unigrama, bigrama, trigrama, 

,analysis,data
"curso de data, dAta science",0,2
tutorial de data analysis,1,1
não sei mais o que escrever analysis,1,0


In [ ]:
tf-idf ~= frequencia do palavra no documento * inverso da frequencia da palavra em todos os documentos

In [137]:
bag_of_words_transformer = TfidfVectorizer()
mx = bag_of_words_transformer.fit_transform(docs).todense()
terms = bag_of_words_transformer.get_feature_names()
pd.DataFrame(mx, columns=terms, index=docs)


# norm = Each output row will have unit norm, either: * ‘l2’: Sum of squares of vector elements is 1.
#The cosine similarity between two vectors is their dot product when l2 norm has been applied. 
#* ‘l1’: Sum of absolute values of vector elements is 1. See preprocessing.normalize.

#use_idf = False 
#use_idf = False e norm = l1, frequência simples

,analysis,curso,data,de,escrever,mais,não,que,science,sei,tutorial
"curso de data, dAta science",0.000000,0.452123,0.687703,0.343851,0.000000,0.000000,0.000000,0.000000,0.452123,0.000000,0.000000
tutorial de data analysis,0.459854,0.000000,0.459854,0.459854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.604652
não sei mais o que escrever analysis,0.322002,0.000000,0.000000,0.000000,0.423394,0.423394,0.423394,0.423394,0.000000,0.423394,0.000000


In [6]:
from nltk.stem import SnowballStemmer

In [139]:
stemmer.stem(docs[2])

'não sei mais o que escrever analys'

In [7]:
stemmer = SnowballStemmer(language='portuguese')
stemmer.stem("analisado"), stemmer.stem("analise")

('analis', 'analis')

In [153]:
bag_of_words_transformer = CountVectorizer(min_df=4, stop_words='english', ngram_range=(3,3))
#ngram range 2,2, 3,3
# stem
mx = bag_of_words_transformer.fit_transform(tweets['txt'])#.todense()
mx.shape

(6706, 618)

In [154]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

In [155]:
for k in range(2,10):
    cluster = make_pipeline(MaxAbsScaler(), KMeans(n_clusters=k, random_state=0))
    cluster.fit(mx)
    p = cluster.predict(mx)
    
    sil = silhouette_score(mx, p)
    print("K = {} - Silhouette: {}".format(k, sil))

K = 2 - Silhouette: 0.7849594690794792
K = 3 - Silhouette: 0.7872649732846221
K = 4 - Silhouette: 0.7899806458433944
K = 5 - Silhouette: 0.7916851880937629
K = 6 - Silhouette: 0.7917029047299742
K = 7 - Silhouette: 0.7967897372340083
K = 8 - Silhouette: 0.7965854032241488
K = 9 - Silhouette: 0.7980160269702951


In [156]:
terms = bag_of_words_transformer.get_feature_names()

In [157]:
k = 9
cluster = make_pipeline(MaxAbsScaler(), KMeans(n_clusters=k, random_state=0))
cluster.fit(mx)
p = cluster.predict(mx)

for c in np.unique(p):
    print("\nCluster {} - Size {}".format(c, (p == c).sum()))
    rank = pd.Series(np.array(mx[p==c].mean(axis=0)).squeeze(), index=terms).sort_values().tail(20)
    print(rank)


Cluster 0 - Size 6453
coronavirus update wuhan                 0.002015
coronavirus death toll                   0.002170
30 hours birth                           0.002325
accidentally leaked real                 0.002479
guan zhuang bing                         0.002634
tencent accidentally leaked              0.002634
zhuang bing du                           0.002634
just 30 hours                            0.002789
novel coronavirus 2019                   0.002944
cruise ship japan                        0.003099
2019 novel coronavirus                   0.003254
news china coronavirus                   0.003409
world health organization                0.003719
coronavirus coronaoutbreak coronanews    0.004184
coronaoutbreak coronanews ncov2019       0.004184
coronavirus asiannetwalking https        0.004339
health coronavirus asiannetwalking       0.004339
amid coronavirus outbreak                0.005114
coronavirus 2019 ncov                    0.005114
coronavirus outbreak https 

In [147]:
centroids = cluster.named_steps['kmeans'].cluster_centers_
for c in range(centroids.shape[0]):
    print("\nCluster {}".format(c))
    rank = pd.Series(centroids[c, :], index=terms).sort_values().tail(20)
    print(rank)


Cluster 0
test           0.058252
lab            0.058252
china          0.058252
experts        0.063107
sars           0.065534
symptoms       0.067961
questions      0.067961
answered       0.067961
biggest        0.067961
efzbdv4cot     0.087379
qvezftkwkq     0.092233
outbreak       0.094660
survive        0.114078
n8owlcko0x     0.169903
virus          0.174757
9orx4j6buu     0.237864
coronavirus    0.260922
gt             0.294498
https          0.588997
details        1.000000
dtype: float64

Cluster 1
amp                    0.015000
case                   0.015846
thanks                 0.016000
good                   0.016308
just                   0.017462
cases                  0.017590
virus                  0.017692
2020                   0.018000
chinese                0.018154
coronavirusoutbreak    0.018308
health                 0.019077
quarantine             0.020000
spread                 0.020615
people                 0.021538
wuhan                  0.022359
lat

In [158]:
cluster.transform(mx)

array([[0.0243835 , 2.2801271 , 3.1933573 , ..., 2.60482705, 1.23336436,
        2.7774603 ],
       [0.0243835 , 2.2801271 , 3.1933573 , ..., 2.60482705, 1.23336436,
        2.7774603 ],
       [0.0243835 , 2.2801271 , 3.1933573 , ..., 2.60482705, 1.23336436,
        2.7774603 ],
       ...,
       [1.73034273, 2.86338604, 3.63284061, ..., 3.12811828, 2.12630845,
        3.27326835],
       [0.0243835 , 2.2801271 , 3.1933573 , ..., 2.60482705, 1.23336436,
        2.7774603 ],
       [0.0243835 , 2.2801271 , 3.1933573 , ..., 2.60482705, 1.23336436,
        2.7774603 ]])

In [159]:
tweets['cluster'] = p

In [160]:
for c in np.unique(p):
    print('Cluster {} = {}'.format(c, tweets[tweets['cluster'] == c]['txt'].iloc[0]))
    print()
    

Cluster 0 = what the actual -

Cluster 1 = What is coronavirus? The biggest questions about the outbreak, answered. SURVIVE SEE DETAILS AT ==&gt;... https://t.co/21AGGnBj58

Cluster 2 = @IsChinar OxyBreath Pro
Highly Effective Anti-Pollution Clean Air Breathing Mask.
Full details please click on a li... https://t.co/uWq9DZ9XI9

Cluster 3 = A 37-year-old woman has become the 15th person in Australia diagnosed with coronavirus - the fifth in Queensland.... https://t.co/UtfPvHwjvR

Cluster 4 = Vals Is Here Surprise That Special Someone Now

You can reach Us On 0205414305or WhatsApp 0555171905 
For The Bes... https://t.co/E9CSZvxFCT

Cluster 5 = 

Can you help us by publishing this link to help raise Awareness of the suffering in #Syria

#Syrie
C'est dur a r... https://t.co/FnS9hjdXFo

Cluster 6 = Did China's Tencent Accidentally Leak The True Terrifying #Coronavirus Statistics https://t.co/gyd8C00Pxg

Cluster 7 = Coronavirus Latest Updates: Everything You Need to Know SEE DETAILS AT ==&gt